In [26]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.decomposition import PCA

In [2]:
df= pd.read_csv('song_lyric_data.csv')

In [92]:
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


In [3]:
X= np.array(df.iloc[:,3])

In [19]:
# This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(X,show_progress_bar=True)

Batches: 100%|██████████| 32/32 [00:08<00:00,  3.60it/s]


In [22]:
embeddings.shape

(1000, 384)

In [20]:
# reducing features for simplifying (95% of variance )
pca = PCA(n_components=0.95)
embeded_lyrics= pca.fit_transform(embeddings)

In [21]:
embeded_lyrics.shape

(1000, 151)

In [81]:
def give_reccom (words,k=2):
    
    # words=np.array(words)
    words_encoded= model.encode(words)
    words_encoded=pca.transform(words_encoded.reshape(1, -1))
    hits = util.semantic_search(words_encoded,embeded_lyrics,top_k=k)
    reccoms_index=[]
    for hit in hits[0]:
        reccoms_index.append(hit['corpus_id'])

    
    return df.iloc[reccoms_index]
    

In [96]:
give_reccom("hi  ",3)

,artist,song,link,text
348,Aiza Seguerra,Huwag Mong Iwan Ang Puso,/a/aiza+seguerra/huwag+mong+iwan+ang+puso_2065...,Kay bilis naman ng panahon \nKailan lang tayo...
359,Aiza Seguerra,Sa Ugoy Ng Duyan,/a/aiza+seguerra/sa+ugoy+ng+duyan_20795061.html,Sana'y di magmaliw ang dati kong araw \nNang ...
353,Aiza Seguerra,Ikaw Na Ang Bahala: Panalangin,/a/aiza+seguerra/ikaw+na+ang+bahala+panalangin...,"Ang ganda ng buhay, mahal kong Diyos \nSana p..."
